# Model Evaluation

Goal: Build and evaluate a chatbot that can carry out a multi-turn conversation, adapt to context, and handle a variety of topics

In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from torch.utils.data import Dataset, DataLoader
import torch
import pandas as pd
# Preprocess the data
from sklearn.model_selection import train_test_split


In [4]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained("t5-small")

df = pd.read_csv("cleaned_conversations.csv")
df.head()


,movie_id,character_1,character_2,conversation_history,history_length,response,genre
0,m0,CAMERON,BIANCA,['[START]'],0,Can we make this quick? Roxanne Korrine and A...,"['comedy', 'romance']"
1,m0,BIANCA,CAMERON,['Can we make this quick? Roxanne Korrine and...,1,"Well, I thought we'd start with pronunciation,...","['comedy', 'romance']"
2,m0,CAMERON,BIANCA,['Can we make this quick? Roxanne Korrine and...,2,Not the hacking and gagging and spitting part....,"['comedy', 'romance']"
3,m0,BIANCA,CAMERON,['Can we make this quick? Roxanne Korrine and...,3,Okay... then how 'bout we try out some French ...,"['comedy', 'romance']"
4,m0,CAMERON,BIANCA,['[START]'],0,You're asking me out. That's so cute. What's ...,"['comedy', 'romance']"


In [14]:
# Define the model save path
model_save_path = "./eval/model_w_val.pth"

# Check if a GPU is available and load the model to GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA is available. Loading model on GPU.")
else:
    device = torch.device('cpu')
    print("CUDA is not available. Loading model on CPU.")

# Load the model
model = T5ForConditionalGeneration.from_pretrained("t5-small")  # Initialize the model
model.load_state_dict(torch.load(model_save_path, map_location=device))  # Load model weights
model.to(device)  # Transfer the model to the specified device (GPU or CPU)
model.eval()  # Put the model in evaluation mode

print(f"Model loaded successfully on {device}.")



CUDA is not available. Loading model on CPU.
Model loaded successfully on cpu.


In [15]:

def preprocess_data(conversation_history, response):
    input_text = " ".join(conversation_history)
    target_text = response
    return input_text, target_text

# Assuming df is your DataFrame with the raw data
df['input'], df['target'] = zip(*df.apply(lambda row: preprocess_data(row['conversation_history'], row['response']), axis=1))

# Split the data into train, validation, and test sets
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [ ]:
class ConversationDataset(Dataset):
    def __init__(self, tokenizer, df, max_length=512):
        self.tokenizer = tokenizer
        self.data = df
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = self.data.iloc[idx]['input']
        target_text = self.data.iloc[idx]['target']

        # Tokenize input and target
        input_encodings = self.tokenizer(input_text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")
        target_encodings = self.tokenizer(target_text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")

        return {
            'input_ids': input_encodings['input_ids'].flatten(),
            'attention_mask': input_encodings['attention_mask'].flatten(),
            'labels': target_encodings['input_ids'].flatten()
        }

# Create datasets
train_dataset = ConversationDataset(tokenizer, train_df)
val_dataset = ConversationDataset(tokenizer, val_df)
test_dataset = ConversationDataset(tokenizer, test_df)

# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)